In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL data/Data_A2/

/content/drive/MyDrive/NNFL data/Data_A2


In [ ]:
%ls -l

total 87
-rw------- 1 root root   259 Apr 29 07:23 class_label.mat
-rw------- 1 root root 70617 Apr 22 07:54 data55.xlsx
-rw------- 1 root root 17039 Apr 29 07:25 data5.xlsx


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

df = pd.read_excel('data5.xlsx', header = None)
df = df.sample(frac=1).reset_index(drop=True)
def sigmoidFunction(Z):
    return (1/(1+np.exp(-Z)), Z)

para=[]
def act1(x, a, b, act):
    if (act == "gaussian"):
        return np.exp(-b*np.linalg.norm(x-a))
    elif (act == "tanh"):
        num = 1-np.exp(-(np.dot(x.T,a)+b))
        den = 1+np.exp(-(np.dot(x.T,a)+b))
        return (num/den)

def init(hiddenLayer, dimensions):
    a = []
    b = []
    for i in range(hiddenLayer):
        a.append(np.random.rand(dimensions,1))
        b.append(np.random.rand(1))
    return (a,b)

def one_hot(y): 
    onehotencoder = OneHotEncoder() 
    y = onehotencoder.fit_transform(y).toarray()
    return y

def compute(hiddenLayer, train_x, test_x, train_y, test_y, act):

    Y_enc = one_hot(train_y)
    H = np.zeros((train_x.shape[0],hiddenLayer))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i]=act1(train_x[j],a[i],b[i],act)
    W = np.dot(np.linalg.pinv(H),Y_enc)
    
    H = np.zeros((test_x.shape[0],hiddenLayer))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i] = act1(test_x[j],a[i],b[i],act)

    p = np.dot(H,W)
    p = np.argmax(p,axis=1)
    p = np.reshape(p,newshape=(p.shape[0],1))
    accuracy = test_y-p
    accuracy = np.sum((accuracy!=0).astype(int))
    return (p.shape[0]-accuracy)/p.shape[0]

l = 256


kf = KFold(n_splits = 5)
X = df.iloc[:, 0:7].values
X = (X - np.mean(X, axis=0))/(np.std(X, axis=0))
Y = df.iloc[:,7].values
Y = np.reshape(Y, newshape=(-1,1))
a, b = init(l, X.shape[1])
kf.get_n_splits(X)
fold = 0
acctemp = 0
overall = 0
for train_index, test_index in kf.split(X):
    fold+=1
    training_data_X = X[train_index]
    training_data_Y = Y[train_index]
    testing_data_X = X[test_index]
    testing_data_Y = Y[test_index]
    acctemp = compute(l, training_data_X, testing_data_X, training_data_Y, testing_data_Y, act = "tanh")
    overall+=acctemp
    print("Fold: ", fold, "Accuracy: ", acctemp)
print("Overall Accuracy (tanh) : ", overall/5, '\n')

kf = KFold(n_splits=5)
X=df.iloc[:,0:7].values
X=(X-np.mean(X,axis=0))/(np.std(X,axis=0))
Y=df.iloc[:,7].values
Y=np.reshape(Y,newshape=(-1,1))
a,b=init(l,X.shape[1])
kf.get_n_splits(X)
fold=0
accuracy=0
overall=0
for train_index, test_index in kf.split(X):
    fold+=1
    training_data_X = X[train_index]
    training_data_Y = Y[train_index]
    testing_data_X = X[test_index]
    testing_data_Y = Y[test_index]
    accuracy = compute(l,training_data_X,testing_data_X,training_data_Y,testing_data_Y,"gaussian")
    overall+= accuracy
    print("Fold: ",fold," Accuracy: ", accuracy)
print("Overall Accuracy (Gaussian) : " , overall/5)

Fold:  1 Accuracy:  0.023809523809523808
Fold:  2 Accuracy:  0.023809523809523808
Fold:  3 Accuracy:  0.023809523809523808
Fold:  4 Accuracy:  0.0
Fold:  5 Accuracy:  0.047619047619047616
Overall Accuracy (tanh) :  0.023809523809523808 

Fold:  1  Accuracy:  0.11904761904761904
Fold:  2  Accuracy:  0.07142857142857142
Fold:  3  Accuracy:  0.14285714285714285
Fold:  4  Accuracy:  0.09523809523809523
Fold:  5  Accuracy:  0.047619047619047616
Overall Accuracy (Gaussian) :  0.09523809523809523
